# Preprocessing REDUCEDHEATARB data

In [ ]:
import pandas as pd
import numpy as np
import pylab as plt
from tqdm.notebook import tqdm

# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

import sys
sys.path.append('../data/')
sys.path.append('../view/')
sys.path.append('../analysis/')

rhc_raw_properties_file='rhc_raw_properties.parquet'

# interpolate__min=1 # interpolation to 1 minute intervals (takes more space & time!)
# rhc_preprocessed_poperties_file='rhc_preprocessed_properties_intv_1_min.parquet'

interpolate__min=5 # default interpolation to 5 minute intervals
rhc_preprocessed_poperties_file='rhc_preprocessed_properties_intv_5_min.parquet'


%load_ext autoreload

%matplotlib inline
%matplotlib widget

from preprocessor import Preprocessor
from plotter import Plot
from nfh_utils import *

meta_df = None


### Load Measured Data Properties from parquet file

In [ ]:
%%time
# Prerequisite: for this example to work, you need to have the b4b_raw_properties.parquet, located in the ../data/ folder.
# One way to get this is to run NeedForHeatExtractionBackup.ipynb, REDUCEDHEATCARB_data_merge.ipynb and REDUCEDHEATCARB_sanity_check.ipynb first,
# but then you have to run this code on the energietransitiewindesheim.nl server

# Attempt to read the Parquet file
try:
    df_prop = pd.read_parquet(
        rhc_raw_properties_file, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")

#sorting the DataFrame index is needed to get good performance on certain filters
#this guarding code to check whether DataFrames are properly sorted
if not df_prop.index.is_monotonic_increasing:
    print('df needed index sorting')
    df_prop = df_prop.sort_index()  

In [ ]:
df_prop.index.unique(level='id').values

In [ ]:
df_prop.index.unique(level='source_category').values

In [ ]:
df_prop.index.unique(level='source_type').values

In [ ]:
%%time
# Rename long source type names
rename_dict = {
    'twomes-co2-occupancy-scd41-m5coreink-firmware': 'living_room',
    'twomes-p1-reader-firmware': 'p1-reader'
}

df_prop = df_prop.rename(index=rename_dict, level='source_type')


In [ ]:
df_prop.index.unique(level='source_type').values

In [ ]:
df_prop

In [ ]:
print(f"df_prop.count().sum(): {df_prop.count().sum():_}")

In [ ]:
df_prop.info()

## Inspecting and preprocessing properties

In [ ]:
df_prop.describe().T

In [ ]:
df_bools_to_float = df_prop[[col for col in df_prop.columns if col.endswith('__bool')]].copy()
for col in df_bools_to_float.columns:
    df_bools_to_float[col] = df_bools_to_float[col].astype('float')
df_bools_to_float.describe().T.drop(columns='count').style.format("{:.2%}")

### Preprocessing temp_out__degC

In [ ]:
prop = 'temp_out__degC'

In [ ]:
df_prop[prop].describe().T

In [ ]:
df_prop

In [ ]:
%matplotlib inline
%matplotlib widget
# Group by 'id' to plot each ID's data separately
grouped = df_prop.groupby('id')[prop]

# Plotting
plt.figure(figsize=(10, 6))
for id_val, group_data in grouped:
    if not group_data.empty and not group_data.eq(0).all():
        group_data.plot.hist(bins=200, alpha=0.5, label=f'ID {id_val}')

plt.title(prop)
plt.xlabel(prop)
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df_prop, meta_df = Preprocessor.filter_min_max(df_prop, prop, min=-28.0, max=40.0, meta_df=meta_df)

In [ ]:
meta_df

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

In [ ]:
df_prop

In [ ]:
# df_prop['remeha_temp_out__degC'].dropna().to_frame(name='remeha_temp_out__degC').unstack(level='id').dropna(axis=1, how='all') # batch_import_remeha_temp_out__degC
df_remeha_temp_out__degC = df_prop.loc[(slice(None), slice(None), 'remeha', slice(None)), 'temp_out__degC'].dropna().unstack(level='id').dropna(axis=1, how='all') # batch_import_remeha_temp_out__degC


In [ ]:
df_remeha_temp_out__degC.count()

In [ ]:
pd.Series(df_remeha_temp_out__degC
          .xs('remeha', level='source_type')
          .xs('batch_import', level='source_category')[458000]
          .dropna()
          .to_frame()
          .index
          .diff()
          ).describe()


In [ ]:
pd.Series(df_remeha_temp_out__degC
          .xs('remeha', level='source_type')
          .xs('batch_import', level='source_category')[478667]
          .dropna()
          .to_frame()
          .index
          .diff()
          ).describe()


In [ ]:
df_remeha_temp_out__degC[458000].dropna().sort_index()

In [ ]:
df_remeha_temp_out__degC[478667].dropna().sort_index()

### Additional weather interpolation checks


In [ ]:
%%time
# Attempt to read the Parquet file
try:
    df_prop = pd.read_parquet(
        rhc_raw_properties_file_path, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")

#sorting the DataFrame index is needed to get good performance on certain filters
#this guarding code to check whether DataFrames are properly sorted
if not df_prop.index.is_monotonic_increasing:
    print('df needed index sorting')
    df_prop = df_prop.sort_index()  

In [ ]:
weather_locations_path = "weather_interpolation_locations.parquet"

In [ ]:
%%time
# read df_weather_locations from Parquet file
try:
    df_weather_locations = pd.read_parquet(
        weather_locations_path, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")

In [ ]:
df_weather_locations

In [ ]:
suspect_df_prop_wind = df_prop.loc[
    (df_prop.index.get_level_values('source_category') == 'batch_import') &
    (df_prop.index.get_level_values('source_type') == 'KNMI') &
    (df_prop['wind__m_s_1'] < 0),
    ['wind__m_s_1']
]

In [ ]:
suspect_wind = suspect_df_prop_wind.count().item()
total_wind = df_prop['wind__m_s_1'].count().item()
print(f"wind: suspect {suspect_wind:_}; total: {total_wind:_}; fraction: {suspect_wind/total_wind:%}")

In [ ]:
suspect_df_prop_wind.index.get_level_values('id').unique()

In [ ]:
suspect_wind_cells = list(df_weather_locations.loc[suspect_df_prop_wind.index.get_level_values('id').unique()]['weather_H3_cell_id'].unique())

In [ ]:
suspect_wind_cells

In [ ]:
suspect_df_prop_wind.describe().T

In [ ]:
suspect_df_prop_sun = df_prop.loc[
    (df_prop.index.get_level_values('source_category') == 'batch_import') &
    (df_prop.index.get_level_values('source_type') == 'KNMI') &
    (df_prop['ghi__W_m_2'] < 0),
    ['ghi__W_m_2']
]

In [ ]:
suspect_sun = suspect_df_prop_sun.count().item()
total_sun = df_prop['ghi__W_m_2'].count().item()
print(f"sun: suspect {suspect_sun:_}; total: {total_sun:_}; fraction: {suspect_sun/total_sun:%}")

In [ ]:
suspect_df_prop_sun.index.get_level_values('id').unique()

In [ ]:
suspect_sun_cells = list(df_weather_locations.loc[suspect_df_prop_sun.index.get_level_values('id').unique()]['weather_H3_cell_id'].unique())

In [ ]:
suspect_df_prop_sun.describe().T

### Preprocessing wind__m_s_1


In [ ]:
prop = 'wind__m_s_1'

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
# Group by 'id' to plot each ID's data separately
grouped = df_prop.groupby('id')[prop]

# Plotting
plt.figure(figsize=(10, 6))
for id_val, group_data in grouped:
    if not group_data.empty and not group_data.eq(0).all():
        group_data.plot.hist(bins=200, alpha=0.5, label=f'ID {id_val}')

plt.title(prop)
plt.xlabel(prop)
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df_prop, meta_df = Preprocessor.filter_min_max(df_prop, prop, min=0.0, meta_df=meta_df)

In [ ]:
meta_df

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

### Preprocessing ghi__W_m_2

In [ ]:
prop = 'ghi__W_m_2'

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
# Group by 'id' to plot each ID's data separately
grouped = df_prop.groupby('id')[prop]

# Plotting
plt.figure(figsize=(10, 6))
for id_val, group_data in grouped:
    if not group_data.empty and not group_data.eq(0).all():
        group_data.plot.hist(bins=200, alpha=0.5, label=f'ID {id_val}')

plt.title(prop)
plt.xlabel(prop)
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df_prop, meta_df = Preprocessor.filter_min_max(df_prop, prop, min=0.0, meta_df=meta_df)

In [ ]:
meta_df

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

### Preprocessing temp_in__degC

In [ ]:
prop = 'temp_in__degC'

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

In [ ]:
df_prop, meta_df = Preprocessor.filter_min_max(df_prop, prop, min=0.0, max=40.0, meta_df=meta_df)

In [ ]:
meta_df

In [ ]:
df_prop[prop].info()

In [ ]:
df_prop[prop].groupby(level='id').count()

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[['temp_in__degC', 'temp_set__degC']].plot.hist(bins=200, alpha=0.5, title = 'temp_in__degC, temp_set__degC')

#### Preprocess indoor temperatures per source_category

In [ ]:
df_prop[prop].groupby(level='source_type', observed=True).describe()

In [ ]:
%matplotlib inline
%matplotlib widget
source_types = df_prop.index.get_level_values('source_type').unique()
valid_source_types = [st for st in source_types if not df_prop.xs(st, level='source_type')[prop].dropna().empty]

# Create the plot
plt.figure(figsize=(10, 6))

for source_type in valid_source_types:
    # Filter DataFrame for the current source_type and drop NaN values
    df_filtered = df_prop.xs(source_type, level='source_type')[prop].dropna()
    
    # Plot the histogram using matplotlib directly
    plt.hist(df_filtered, bins=200, alpha=0.5, density=True, label=source_type)

plt.legend()
plt.show()

In [ ]:
source_types

In [ ]:
df_prop.index.dtypes

In [ ]:
df_prop.info()

In [ ]:
%autoreload 2 
df_calibration_factors = Preprocessor.compute_calibration_factors(df_prop, prop, 'living_room', 'remeha', 20)

In [ ]:
df_calibration_factors.set_index('id').T

In [ ]:
pd.DataFrame(df_calibration_factors.mean()).T

In [ ]:
%%time
df_prop, metadata = Preprocessor.create_calibrated_property(df_prop, prop, 'living_room', 'remeha', 20)        


In [ ]:
df_prop[prop].groupby(level='source_type', observed=True).describe()

In [ ]:
metadata

In [ ]:
%matplotlib inline
%matplotlib widget
source_types = df_prop.index.get_level_values('source_type').unique()
valid_source_types = [st for st in source_types if not df_prop.xs(st, level='source_type')[prop].dropna().empty]

# Create the plot
plt.figure(figsize=(10, 6))

for source_type in valid_source_types:
    # Filter DataFrame for the current source_type and drop NaN values
    df_filtered = df_prop.xs(source_type, level='source_type')[prop].dropna()
    
    # Plot the histogram using matplotlib directly
    plt.hist(df_filtered, bins=200, alpha=0.5, density=True, label=source_type)

plt.legend()
plt.show()

### Preprocessing temp_sup__degC

In [ ]:
prop = 'temp_sup__degC'

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

In [ ]:
df_prop, meta_df = Preprocessor.filter_min_max(df_prop, prop, min=0.0, max=100.0, meta_df=meta_df)

In [ ]:
meta_df

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

In [ ]:
df_prop['meter_code__str'].unique()

In [ ]:
df_prop['dsmr_version__0'].unique()

In [ ]:
df_prop.groupby(['id', 'meter_code__str', 'dsmr_version__0']).size().reset_index(name='count').set_index(['id', 'meter_code__str', 'dsmr_version__0'])

### Preprocessing temp_ret__degC

In [ ]:
prop = 'temp_ret__degC'

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

In [ ]:
df_prop, meta_df = Preprocessor.filter_min_max(df_prop, prop, min=0.0, max=100.0, meta_df=meta_df)

In [ ]:
meta_df

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[['temp_sup__degC', 'temp_ret__degC']].plot.hist(bins=200, alpha=0.5, title = 'temp_sup__degC, temp_ret__degC')

### Preprocessing dhw_temp_out__degC

In [ ]:
prop = 'dhw_temp_out__degC'

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

In [ ]:
df_prop, meta_df = Preprocessor.filter_min_max(df_prop, prop, min=0.0, max=100.0, meta_df=meta_df)

In [ ]:
meta_df

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

### Preprocessing dhw_flow__l_min_1

In [ ]:
prop = 'dhw_flow__l_min_1'

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

In [ ]:
df_prop[prop].info()

In [ ]:
df_prop[prop].groupby(level='id').describe().style.map(Preprocessor.highlight_zero, subset=['std'])

In [ ]:
df_prop, meta_df = Preprocessor.filter_id_prop_with_std_zero(df_prop, prop, meta_df=meta_df)

In [ ]:
meta_df

In [ ]:
df_prop[prop].groupby(level='id').describe().style.map(Preprocessor.highlight_zero, subset=['std'])

In [ ]:
df_prop[prop].describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

In [ ]:
df_prop_filtered = df_prop[prop][df_prop[prop] != 0]

In [ ]:
df_prop_filtered.info()

In [ ]:
df_prop_filtered.describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop_filtered.plot.hist(bins=200, alpha=0.5, title = prop)

## Inspecting and preprocessing parameters, i.e. constant (?) properties


### Preprocessing temp_ch_sup_max__degC

In [ ]:
df_prop['temp_ch_sup_max__degC'].groupby(level='id').describe().T

### Inspecting power_ch_max__kW

In [ ]:
prop = 'power_ch_max__kW'

In [ ]:
df_prop[prop].groupby(level='id').describe().T

## Inspecting and preprocessing smart meter values

In [ ]:
use_meter_cols = ['e_use_hi_cum__kWh', 'e_use_lo_cum__kWh']
ret_meter_cols = ['e_ret_hi_cum__kWh', 'e_ret_lo_cum__kWh']
all_meter_cols = use_meter_cols + ret_meter_cols

In [ ]:
prop = all_meter_cols + ['dsmr_version__0']

In [ ]:
df_prop['dsmr_version__0'].unique()

In [ ]:
df_prop[prop].describe()

In [ ]:
df_prop[prop].groupby(level='id').describe().T

In [ ]:
%%time
df_prop, meta_df = Preprocessor.filter_electricity_meter_values(df_prop, min_valid_cum__kWh=15.0, meta_df=meta_df)

In [ ]:
meta_df

In [ ]:
df_prop['dsmr_version__0'].unique()

In [ ]:
df_prop[prop].describe()

In [ ]:
df_prop[prop].groupby(level='id').describe().T

#### Inspecting and preprocessing g_use_cum__m3

In [ ]:
prop = 'g_use_cum__m3'

In [ ]:
df_prop[prop].describe()

In [ ]:
df_prop[prop].groupby(level='id').describe().sort_values(by='std', ascending=False).T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=400, alpha=0.5, title = prop)

#### Inspecting and preprocessing meter_code__str

In [ ]:
prop = 'meter_code__str'

In [ ]:
df_prop[prop].unique()

In [ ]:
df_metercodes = df_prop.groupby(['id', 'meter_code__str', 'dsmr_version__0']).size().reset_index(name='count').set_index(['id', 'meter_code__str', 'dsmr_version__0'])

In [ ]:
df_metercodes

In [ ]:
id_lists = (df_metercodes
            .sort_values(by='dsmr_version__0')
            .reset_index()
            .groupby('dsmr_version__0')['id']
             .apply(list))

In [ ]:
id_lists

In [ ]:
# Print the full content of the Series
for key, value in id_lists.items():
    print(f"dsmr_version: {key:.1f}, {len(value)} ids: {sorted(value)}")

# Inspect number of null measurements and time covered before interpolation

In [ ]:
%%time
count_non_null_before_interpolation = Preprocessor.count_non_null_measurements(df_prop).sort_index(axis=1).sort_values(by='total', ascending=False)

In [ ]:
count_non_null_before_interpolation.T.style.map(Preprocessor.highlight_zero)

In [ ]:
%%time
covered_time_before_interpolation = Preprocessor.calculate_covered_time(df_prop).sort_index(axis=1).sort_values(by='total', ascending=False)

In [ ]:
covered_time_before_interpolation.sort_values(by='remeha_temp_out__degC', ascending=False).T.style.map(Preprocessor.highlight_zero).format(lambda x: f'{x.days} d')

## Plotting data using in analysis

In [ ]:
units_to_mathtext = {
    'ppm' : r'$ppm$',
    'kWh' : r'$kWh$',
    'm3' : r'$m^{3}$',
    'degC' : r'$°C$',
    'W' : r'$W$',
    'V' : r'$V$',
    '0' : r'$[-]$',
    'bool': r'$0 = False; 1 = True$',
    'W_m_2' : r'$W\cdotm^{-1}$'
}

In [ ]:
# %%time

# #Plot all properties with a single unit for a single id
# Plot.dataframe_properties_plot(df_prop.loc[[948634, 999169]][[prop for prop in df_prop.columns.values if prop.split('__')[-1] in['ppm', '0', 'bool']]], units_to_mathtext)

In [ ]:
# props = [prop for prop in df_prop.columns.values if prop.split('__')[-1] in['ppm', '0', 'bool', 'p']]
props = ['co2__ppm', 'occupancy__p', 'onboarded__p']

In [ ]:
props

In [ ]:
df_prop[props].info()

In [ ]:
df = df_prop[props] 

In [ ]:
df.describe().T


In [ ]:
stats = df.groupby(level=['source_category', 'source_type','id'], observed=True).describe().transpose()
stats.columns = stats.columns.swaplevel(0,1)
stats = stats.sort_index(axis=1, level=0)
stats

In [ ]:
%matplotlib inline
%matplotlib widget
prop = 'co2__ppm'
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

In [ ]:
%matplotlib inline
%matplotlib widget
prop = 'ch_water_pump_speed__0'
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

In [ ]:
%matplotlib inline
%matplotlib widget
prop = 'dhw_flow__l_min_1'
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

## Preprocessing co2__ppm
Filtering out measurement errors of 0 ppm below 5 ppm

In [ ]:
prop = 'co2__ppm'

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)

In [ ]:
df_prop[prop].groupby(level=['id']).describe().T

#### No need to filter out values below 5 ppm (which would be clear measurement errors)

In [ ]:
df_prop, meta_df = Preprocessor.filter_min_max(df_prop, prop, min=5, meta_df=meta_df)

In [ ]:
meta_df

#### Check to see whether minimum changed

In [ ]:
df_prop[prop].groupby(level=['id']).describe().T

In [ ]:
%matplotlib inline
%matplotlib widget
df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)


## After preprocessing, before temporal interpolation

In [ ]:
meta_df

In [ ]:
meta_df.to_excel('rhc_preprocessing.xlsx')

In [ ]:
df_prop.size

In [ ]:
print(f"df_prop.count().sum(): {df_prop.count().sum():_}")

In [ ]:
df_prop.info()

# Temporal interpolation of properties


In [ ]:
%%time 

# Define the properties and their respective limit__min values
property_limits = {
    'boiler_status_blocking_mode__bool': 5,
    'boiler_status_burner_start__bool': 5,
    'boiler_status_burner_stop__bool': 5,
    'boiler_status_burning_ch__bool': 5,
    'boiler_status_burning_dhw__bool': 5,
    'boiler_status_controlled_stop__bool': 5,
    'boiler_status_de_air__bool': 5,
    'boiler_status_heat_demand__bool': 5,
    'boiler_status_locking_mode__bool': 5,
    'boiler_status_pump_post_run__bool': 5,
    'boiler_status_standby__bool': 5,
    'dhw_flow__l_min_1': 5,
    'dhw_temp_out__degC': 5,
    'gas_valve_closed__bool': 5,
    'gas_valve_open__bool': 5,
    'temp_sup__degC': 5,
    'temp_ret__degC': 5,
    'fan_rotations__min_1': 5,
    'batch_import_KNMI_temp_out__degC': 4 * 60, # 4 hours
    'batch_import_KNMI_ghi__W_m_2': 4 * 60, # 4 hours
    'batch_import_KNMI_wind__m_s_1': 4 * 60, # 4 hours
    'batch_import_KNMI_air__Pa': 4 * 60, # 4 hours
    'batch_import_KNMI_air_rel_humidity__0': 4 * 60, # 4 hours
    'temp_ch_supmax__degC': 2 * 24 * 60,  # 2 days in minutes
    'power_ch_max__kW': 2 * 24 * 60,     # 2 days in minutes
    'ch_set_fan_rotations_max__min_1': 2 * 24 * 60,  # 2 days in minutes
    'ch_set_fan_rotations_min__min_1': 2 * 24 * 60,   # 2 days in minutes
    'e_ret_monthly_hi_cum__kWh': 32 * 24 * 60,   # 32 days in minutes,
    'e_ret_monthly_lo_cum__kWh': 32 * 24 * 60,   # 32 days in minutes,
    'e_use_monthly_hi_cum__kWh': 32 * 24 * 60,   # 32 days in minutes,
    'e_use_monthly_lo_cum__kWh': 32* 24 * 60,   # 32 days in minutes,
    'g_use_monthly_cum__m3': 32 * 24 * 60,   # 32 days in minutes,
}

In [ ]:
# properly sort and index dataframe to prevent performance warnings
df_prop = df_prop.sort_index()

In [ ]:
df_prop.info()

In [ ]:
print(f"df_prop.count().sum(): {df_prop.count().sum():_}")

In [ ]:
# Create overview of non-null counts per column, source_category, and source_type
df_prop.groupby(['source_category', 'source_type'], observed=True).apply(lambda x: x.notna().sum()).reset_index().T

## Inspect measurement intervals

In [ ]:
%autoreload 2
df_intervals = Preprocessor.analyze_intervals(df_prop, default_limit__min=90, property_limits=property_limits, interpolate__min=5)

In [ ]:
def mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None

In [ ]:
# Group by 'id', 'source_category', and 'source_type' and calculate descriptive statistics
# (df_intervals.groupby(['id', 'source_category', 'source_type'], observed=True)
(df_intervals.groupby(['source_category', 'source_type'], observed=True)
 .agg({'modal_intv__min': ['mean', 'std', 'min', 'max', mode],
    'limit__min': ['mean', 'std', 'min', 'max', mode],
    'upsample__min': ['mean', 'std', 'min', 'max', mode],
    'interpolate__min': ['mean', 'std', 'min', 'max', mode],
    'limit': ['mean', 'std', 'min', 'max', mode]
})).T

In [ ]:
# Slice df_intervals to show only rows where source_category is 'batch_import'
df_intervals.loc[(slice(None), 'batch_import', slice(None)), :].sort_values(by='upsample__min', ascending=False)


In [ ]:
df_intervals.loc[(slice(None), 'batch_import', slice(None), 'temp_out__degC'), :]

In [ ]:
df_intervals.loc[(slice(None), 'cloud_feed', slice(None), 'g_use_cum__m3'), :]

In [ ]:
df_intervals.loc[(slice(None), 'device', slice(None), 'g_use_cum__m3'), :].sort_values(by='modal_intv__min', ascending=False)

In [ ]:
id_lists = (df_intervals
 .loc[(slice(None), 'device', slice(None), 'g_use_cum__m3'), :]
 .sort_values(by='modal_intv__min', ascending=False)
 .reset_index()
 .groupby('modal_intv__min')['id']
 .apply(list))

# Print the full content of the Series
for key, value in id_lists.items():
    print(f"modal interval (minutes): {key}, {len(value)} ids: {sorted(value)}")

The IDs with a modal interval of 60 minutes correspond neatly to the IDs that have a smart meter that adheres to DSMR 4.2, which only support a gas meter reading once per hour

## Perform temporal interpolation

In [ ]:
%%time
%autoreload 2

# Perform interpolation
df_interpolated = Preprocessor.interpolate_time(df_prop=df_prop,
                                                default_limit__min=90,
                                                property_limits=property_limits,
                                                interpolate__min=interpolate__min,
                                                restore_original_types=True,
                                                inplace=False)

In [ ]:
df_interpolated

In [ ]:
print(f"df_interpolated.count().sum(): {df_interpolated.count().sum():_}")

In [ ]:
df_interpolated.info()

In [ ]:
df_interpolated.index.dtypes

In [ ]:
# Create overview of non-null counts per column, source_category, and source_type
df_interpolated.groupby(['source_category', 'source_type'], observed=True).apply(lambda x: x.notna().sum()).reset_index().T

In [ ]:
df_prop.describe().T

In [ ]:
df_interpolated.describe().T.sort_index()

In [ ]:
df_bools_to_float = df_interpolated[[col for col in df_interpolated.columns if col.endswith('__bool')]].copy()
for col in df_bools_to_float.columns:
    df_bools_to_float[col] = df_bools_to_float[col].astype('float')
df_bools_to_float.describe().T.drop(columns='count').style.format("{:.2%}")

# Inspect number of null measurements and time covered after interpolation

In [ ]:
count_non_null_after_interpolation = Preprocessor.count_non_null_measurements(df_interpolated).sort_index(axis=1).sort_values(by='total', ascending=False)

In [ ]:
count_non_null_after_interpolation.T.style.map(Preprocessor.highlight_zero)

In [ ]:
mandatory_props=['living_room_calibrated_temp_in__degC', 'p1-reader_g_use_cum__m3']
filter_props = mandatory_props + ['all_mandatory_props']

In [ ]:
covered_time_after_interpolation = Preprocessor.calculate_covered_time(df_interpolated, mandatory_props=mandatory_props).sort_values(by='all_mandatory_props', ascending=False)[filter_props]

In [ ]:
covered_time_after_interpolation.T.style.map(Preprocessor.highlight_zero).format(lambda x: f'{x.days} d')

In [ ]:
covered_time_after_interpolation.to_excel('rhc_covered_time.xlsx')

In [ ]:
%%time
covered_time_after_interpolation = Preprocessor.calculate_covered_time(df_interpolated).sort_index(axis=1).sort_values(by='total', ascending=False)

In [ ]:
covered_time_after_interpolation.T.style.map(Preprocessor.highlight_zero).format(lambda x: f'{x.days} d')

# Converting raw properties dataframe to preprocessed dataframe


In [ ]:
%%time
%autoreload 2
df_prep = Preprocessor.unstack_source_cat_and_type(df_interpolated)

In [ ]:
df_prep.info()

In [ ]:
df_prep.index.dtypes

In [ ]:
df_prep

# Convert cumulative smart meter values to average power

In [ ]:
meter_props = sorted([item for item in list(df_prep.columns) if item.endswith('_cum__kWh') or item.endswith('_cum__m3')])

In [ ]:
%%time
# TODO: update heating value to convert P1 gas m3 to average power. 
%autoreload 2
df_prep = Preprocessor.convert_cumulative_to_avg_power(df_prep,
                                                       props=meter_props,
                                                       heating_value__MJ_m_3=(gas_groningen_nl_avg_std_hhv__J_m_3 / 1e6),
                                                       heating_value_name__str='hhv'
                                                      )

In [ ]:
power_props = sorted([item for item in list(df_prep.columns) if item.endswith('__W')])

In [ ]:
print(df_prep.describe().T.to_string())

# Writing preprocessed interpolated properties to a parquet file

In [ ]:
df_prep.info()

In [ ]:
df_prep.index.dtypes

In [ ]:
%%time 
df_prep.to_parquet(rhc_preprocessed_poperties_file, index=True, engine='pyarrow')

# Missing value overview

In [ ]:
print(f"df_prep.count().sum(): {df_prep.count().sum():_}")

In [ ]:
list(sorted(df_prep.columns))

In [ ]:
mandatory_sourceprops =  ['batch_import_KNMI_temp_out__degC', 
                          'batch_import_KNMI_ghi__W_m_2',
                          'batch_import_KNMI_wind__m_s_1',
                          'batch_import_KNMI_air__Pa',
                          'batch_import_remeha_temp_in__degC',
                          'batch_import_remeha_temp_ret__degC',
                          'batch_import_remeha_temp_sup__degC',
                          'device_p1-reader_g_use_cum__m3'
                         ]

In [ ]:
Plot.plot_missing_data_overview(df_prep, properties_include=mandatory_sourceprops, freq='1W', title_fontsize=8)

In [ ]:
Plot.plot_missing_data_overview(df_prep,
                                properties_include=['batch_import_KNMI_temp_out__degC',
                                                    'batch_import_KNMI_ghi__W_m_2',
                                                    'batch_import_KNMI_wind__m_s_1',
                                                    'batch_import_KNMI_air__Pa',
                                                    'batch_import_KNMI_air_rel_humidity__0',                                                   ],
                                freq='1W', 
                                title_fontsize=8)

In [ ]:
Plot.plot_missing_data_overview(df_prep, properties_include=['device_p1-reader_g_use_cum__m3'], freq='1W', title_fontsize=8)

In [ ]:
Plot.plot_missing_data_overview(df_prep, properties_include=['batch_import_remeha_temp_out__degC'], freq='1W', title_fontsize=8)

In [ ]:
Plot.plot_missing_data_overview(df_prep, properties_include=['batch_import_KNMI_temp_out__degC',
                                                             'batch_import_KNMI_ghi__W_m_2',
                                                             'batch_import_KNMI_wind__m_s_1',
                                                             'batch_import_remeha_temp_in__degC',
                                                             'batch_import_remeha_temp_ret__degC',
                                                             'batch_import_remeha_temp_sup__degC',
                                                             # 'batch_import_remeha_g_use_ch_lhv__W',
                                                             'device_p1-reader_e_use_hi_cum__kWh', 
                                                             'device_p1-reader_e_use_lo_cum__kWh',
                                                             'device_living_room_co2__ppm',
                                                             'device_living_room_occupancy__p', 
                                                             'device_living_room_onboarded__p',
                                                             'device_living_room_rel_humidity__0', 
                                                             'device_living_room_temp_in__degC',
                                                             'device_p1-reader_g_use_cum__m3' 
                                                            ], 
                                freq='1W', title_fontsize=6)

In [ ]:
Plot.plot_missing_data_overview(df_prep, properties_include=['device_p1-reader_e_use_hi_cum__kWh', 
                                                             'device_p1-reader_e_use_lo_cum__kWh',
                                                             'device_living_room_co2__ppm',
                                                             'device_living_room_occupancy__p', 
                                                             'device_living_room_onboarded__p',
                                                             'device_living_room_rel_humidity__0', 
                                                             'device_living_room_temp_in__degC',
                                                             'device_p1-reader_g_use_cum__m3' 
                                                            ], 
                                freq='1W', title_fontsize=8)

In [ ]:
Plot.plot_missing_data_overview(df_prep, properties_include=['batch_import_remeha_temp_in__degC'], freq='1W', title_fontsize=8)

In [ ]:
Plot.plot_missing_data_overview(df_prep, properties_include=['device_living_room_co2__ppm',
                                                             'device_living_room_occupancy__p', 
                                                             'device_living_room_onboarded__p',
                                                             'device_living_room_rel_humidity__0', 
                                                             'device_living_room_temp_in__degC'
                                                            ], freq='1W', title_fontsize=8)

In [ ]:
Plot.plot_missing_data_overview(df_prep, properties_include=['device_living_room_calibrated_temp_in__degC'], freq='1W', title_fontsize=8)

In [ ]:
Plot.plot_missing_data_overview(df_prep, properties_include=['batch_import_remeha_temp_in__degC', 'device_living_room_calibrated_temp_in__degC'], freq='1W', title_fontsize=8)

In [ ]:
Plot.plot_missing_data_overview(df_prep, properties_include=['cloud_feed_enelogic_g_use_cum__m3'], freq='1W', title_fontsize=8)

In [ ]:
Plot.plot_missing_data_overview(df_prep, properties_include=['batch_import_enelogic_g_use_monthly_cum__m3'], freq='1M', title_fontsize=8)

## Alternative plot: using colored axes


In [ ]:
%%time
Plot.plot_data_availability(df_prep, properties_include=mandatory_sourceprops, alpha=0.5, figsize=(12, 12), title_fontsize=8)


In [ ]:
%%time
Plot.plot_data_availability(df_prep, properties_include=['cloud_feed_enelogic_g_use_cum__m3'], alpha=0.5, figsize=(12, 12), title_fontsize=8)


# Histograms of streak length


In [ ]:
df_prep.info()

In [ ]:
Preprocessor.calculate_streak_durations(df_prep, mandatory_sourceprops)

In [ ]:
Preprocessor.calculate_streak_durations(df_prep, ['cloud_feed_enelogic_g_use_cum__m3'])

In [ ]:
Preprocessor.calculate_streak_durations(df_prep, ['device_p1-reader_g_use_cum__m3'])

In [ ]:
Preprocessor.calculate_streak_durations(df_prep, ['device_living_room_co2__ppm'])

# Plotting results: time series before and after preprocessing

In [ ]:
%autoreload 2
units_to_mathtext = property_types = {
    'degC' : r'$°C$',
    'ppm' : r'$ppm$',
    '0' : r'$[-]$',
    'bool': r'$0 = False; 1 = True$',
    'p' : r'$persons$'
}

In [ ]:
# %%time
# #Plot all properties with a single unit for a single id
# Plot.dataframe_properties_plot(df_prop.loc[[401632]][[prop for prop in df_prop.columns.values if prop.split('__')[-1] == 'm3']], units_to_mathtext)

In [ ]:
# %%time
# #Plot all preprocessed properties with a single unit for a single id
# Plot.dataframe_preprocessed_plot(df_prep.loc[[401632]][[prop for prop in df_prep.columns.values if prop.split('__')[-1] == 'degC']], units_to_mathtext)

In [ ]:
# TO DO: add prop_ and prep_ in a `prop-prep` column; merge into single dataframe and unstack, thus allowing for close inspection of preprocessed data

# Other examples: temp_in__degC, minmax filtering and using the static outlier filter, per id
Filtering out extreme temperatures based on mean and standard deviation per room

In [ ]:
prop = 'temp_in__degC'

In [ ]:
%matplotlib inline
%matplotlib widget

df_prop[prop].plot.hist(bins=200, alpha=0.5, title = prop)
